In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import string
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Masking
from keras.layers import Attention, LayerNormalization, Dropout
from keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir("/content/drive/My Drive/Project3/")
os.getcwd()


'/content/drive/My Drive/Project3'

In [4]:
df=pd.read_csv('Data.csv')

In [5]:

df

,english,sinhala
0,you will receive a package in the mail,ඔයාට තැපෑලෙන් පැකේජ් එකක් හම්බවේවි
1,you will receive a confirmation code after com...,ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කි...
2,you will receive a discount on your next purchase,ඊලඟ මිලදී ගැනීම කරන කොට ඔයාට වට්ටමක් හම්බවේවි
3,you will receive a phone call from our custome...,ඔයාට අපේ පාරිභෝගික සේවා කණ්ඩායමෙන් දුරකථන ඇමතු...
4,you will receive a notification when your orde...,ඔයාගේ ඇණවුම සූදානම් උනාට පස්සෙ ඔයාට දැනුම් දීම...
...,...,...
80679,Deep in the enchanted forest a group of woodla...,වශීකෘත වනාන්තරයේ ගැඹුරින් නුවණැති මහලු බකමූණෙක...
80680,On a sunny summer day a group of friends gathe...,අව්ව සහිත ගිම්හාන දිනයක මිතුරන් පිරිසක් වැලි ම...
80681,In a quaint seaside village a mysterious stran...,විචිත්‍රවත් මුහුදු වෙරළේ ගම්මානයකට අද්භූත ආගන්...
80682,Thomas an aspiring writer found inspiration in...,අභිලාෂකාමී ලේඛකයෙකු වන තෝමස් සෑම අස්සක් මුල්ලක...


In [6]:
spa_sentences = df["sinhala"] + "\t" + df["english"]

# Save the concatenated sentences to a text file
with open("spa.txt", "w", encoding="utf-8") as file:
    for sentence in spa_sentences:
        file.write(str(sentence) + "\n")


In [7]:
text_file ="spa.txt"

In [8]:
text_file ="spa.txt"
with open(text_file) as f:
  lines=f.read().split("\n")[:-1]

In [9]:
i=0
for line in lines:
  print(line)
  i=i+1
  if(i==20):
    break

ඔයාට තැපෑලෙන් පැකේජ් එකක් හම්බවේවි	you will receive a package in the mail
ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කිරීමේ කේතයක්  හම්බවේවි	you will receive a confirmation code after completing the registration
ඊලඟ මිලදී ගැනීම කරන කොට ඔයාට වට්ටමක් හම්බවේවි	you will receive a discount on your next purchase
ඔයාට අපේ පාරිභෝගික සේවා කණ්ඩායමෙන් දුරකථන ඇමතුමක් හම්බවේවි	you will receive a phone call from our customer service team
ඔයාගේ ඇණවුම සූදානම් උනාට පස්සෙ ඔයාට දැනුම් දීමක් හම්බවේවි	you will receive a notification when your order is ready for pickup
ඔයාට පැය 24ක් ඇතුලත ඔයාගෙ විමසීමට පිළිතුරක්  හම්බවේවි	you will receive a response to your inquiry within 24 hours
ඔයාට ඔයාගේ පක්ෂපාතිත්වය වෙනුවෙන් තෑග්ගක් හම්බවේවි	you will receive a gift for your loyalty
ඔයාට උත්සවයට ආරාධනාවක් හම්බවේවි	you will receive an invitation to the event
ඔයාට රිටන් කරන භාණ්ඩය වෙනුවෙන් ආපසු මුදල් ගෙවීමක් හම්බවේවි	you will receive a refund for the returned item
ඔයාට විස්තර එක්ක  ඊමේල් එකක් හම්බවේවි	you'll receive an e

In [10]:
import random
import string
import re

text_pairs = []


for line in lines:
    if '\t' in line:
        english, sinhala = line.split("\t")
        sinhala = "[start]" + sinhala.strip() + "[end]"
        text_pairs.append((english.strip(), sinhala))
for i in range(3):
    print(random.choice(text_pairs))


('මම අඳින්න කැමති හැමදේම ඇන්දා', '[start]i wore everything i wanted to wear[end]')
('ඒක අවුලක් නැතිවේවි', '[start]it will be fine[end]')
('මොකද විකුණන කෙනා එයාව අඳුරන නිසා', '[start]because the seller knows him[end]')


In [11]:
import random
random.shuffle(text_pairs)


In [12]:
num_val_samples =int(0.15*len(text_pairs))
num_train_samples=len(text_pairs) -2* num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [13]:
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))

Total sentences: 80633
Training set size: 56445
Validation set size: 12094
Testing set size: 12094


In [14]:
len(train_pairs)+len(val_pairs)+len(test_pairs)


80633

In [15]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [16]:
f"[{re.escape(strip_chars)}]"


'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [17]:
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
    lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length + 1,
  standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]


In [18]:
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [19]:
batch_size=64

def format_dataset(eng,spa):
  eng=source_vectorization(eng)
  spa=target_vectorization(spa)
  return({
      "english":eng,
      "sinhala":spa[:,:-1],
  },spa[:,1:])

def make_dataset(pairs):
  eng_texts, spa_texts =zip(*pairs)
  eng_texts =list(eng_texts)
  spa_texts =list(spa_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(format_dataset, num_parallel_calls=4)
  return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

for inputs, targets in train_ds.take(1):
  print(f"inputs['english'].shape:{inputs['english'].shape}")
  print(f"inputs['sinhala'].shape:{inputs['sinhala'].shape}")
  print(f"targets.shape:{targets.shape}")

  inputs['english'].shape:  (64, 20)
  inputs['sinhala'].shape: (64, 20)
  targets.shape: (64, 20)
print(list(train_ds.as_numpy_iterator())[50])

inputs['english'].shape:(64, 20)
inputs['sinhala'].shape:(64, 20)
targets.shape:(64, 20)
({'english': array([[    1,  9593, 10876, ...,     0,     0,     0],
       [    5,   916,    37, ...,     0,     0,     0],
       [   76,   153,     1, ...,     0,     0,     0],
       ...,
       [    6,     1,   789, ...,     0,     0,     0],
       [ 4597,     0,     0, ...,     0,     0,     0],
       [   47,     0,     0, ...,     0,     0,     0]]), 'sinhala': array([[    1,     1,  5341, ...,     0,     0,     0],
       [    7,   347,   181, ...,     0,     0,     0],
       [  600,    43,  4240, ...,     0,     0,     0],
       ...,
       [  340,     3, 13030, ...,     0,     0,     0],
       [   63,  2524,     0, ...,     0,     0,     0],
       [  741,     0,     0, ...,     0,     0,     0]])}, array([[    1,  5341,     0, ...,     0,     0,     0],
       [  347,   181,     8, ...,     0,     0,     0],
       [   43,  4240,   466, ...,     0,     0,     0],
       ...,
      

In [20]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config


In [21]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask
        )
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [22]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [23]:
embed_dim = 256
dense_dim = 2048
num_heads = 8


# Define the encoder inputs
encoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="english")

# Add positional embedding to the encoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

# Encode the inputs using TransformerEncoder
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# Define the decoder inputs
decoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="sinhala")

# Add positional embedding to the decoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

# Decode the inputs using TransformerDecoder
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)

# Apply dropout
x = layers.Dropout(0.5)(x)

# Generate decoder outputs
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

# Create the transformer model
transformer = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

In [24]:
transformer.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])

transformer.fit(train_ds, epochs=30, validation_data=val_ds)


Epoch 1/30
882/882 [==============================] - 79s 78ms/step - loss: 5.0069 - accuracy: 0.2963 - val_loss: 4.1254 - val_accuracy: 0.3714
Epoch 2/30
882/882 [==============================] - 64s 73ms/step - loss: 4.1652 - accuracy: 0.3864 - val_loss: 3.7698 - val_accuracy: 0.4196
Epoch 3/30
882/882 [==============================] - 61s 69ms/step - loss: 3.7989 - accuracy: 0.4347 - val_loss: 3.5655 - val_accuracy: 0.4485
Epoch 4/30
882/882 [==============================] - 65s 73ms/step - loss: 3.5344 - accuracy: 0.4689 - val_loss: 3.4840 - val_accuracy: 0.4589
Epoch 5/30
882/882 [==============================] - 61s 69ms/step - loss: 3.3337 - accuracy: 0.4969 - val_loss: 3.4452 - val_accuracy: 0.4674
Epoch 6/30
882/882 [==============================] - 61s 69ms/step - loss: 3.1754 - accuracy: 0.5219 - val_loss: 3.4282 - val_accuracy: 0.4800
Epoch 7/30
882/882 [==============================] - 61s 69ms/step - loss: 3.0487 - accuracy: 0.5414 - val_loss: 3.4863 - val_accuracy:

In [25]:
# Define the vocabulary and index lookup for Spanish
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))


-
නිකම්ම නින්ද යනකල් කියන කතාවක් විතරයි ඔයාගෙ තාත්තා හදපු
[start] just [UNK] for [UNK] your story is a job[end]           
-
මම කියන්නේ
[start] say[end]                   
-
හායි
[start] really[end]                   
-
ලේඩිබග්     ඔව්
[start] yes[end]                   
-
ලස්සන කාර් එකක්  රූපවාහිණියක් ලොකු එකක්
[start] from a phone is a very strong name[end]            
-
ඒ වෙනුවට කරේ මට සෙරෙප්පු පාරවල් දීපු එක
[start] for him i want to fight and i do is make for three choice[end]      
-
ඔයාගෙ සල්ලි අපේ නියෝජිතයා ගවට ගිය ගමන්ම
[start] from your money and meet the way[end]             
-
ස්තූතියි
[start] from you[end]                  
-
බායි ඔයාලාව හමුවීම සතුටක්
[start] from you meet you[end]                
-
ඔබේ හැම කරදරයක්ම දියවෙනවා
[start] from your [UNK]                 
-
ඔයා කොහොමද මෙහෙම කරන්නේ
[start] how do you do this[end]               
-
වැඩේ හරි දැන් අන්තිම ප්‍රාර්ථනාව
[start] the right now last wish[end]               
-
අපි විහිළුවක් කරේ
[start] from kidding [UNK]     

In [26]:
# Define the vocabulary and index lookup for Sinhala
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print("English: ", input_sentence)
    print("Sinhala: ", decode_sequence(input_sentence))

-
English:  එයා සාමදානයෙන් සැතපේවා මේරි කෙලි
Sinhala:  [start] from him as the name is [UNK] mary[end]            
-
English:  ගිය බ්‍රහස්පතින්දා
Sinhala:  [start] last night[end]                  
-
English:  සිද්ධාන්ත පරීක්ෂාවට පැමිණි
Sinhala:  [start] from [UNK]                  
-
English:  අමතක කරන්න එපා ඔයාගෙ අළුත් ඥාතීන්ට මේ ගැන දැනුම් දෙන්න අපේ අවසරය
Sinhala:  [start] forget your not [UNK] new [UNK] and this new [UNK]          
-
English:  කෙහෙල්මල් මොමොමොන් එකක් ඇතුලට ඔබනවාද කියලා   "
Sinhala:  [start] from [UNK] there is in[end]               
-
English:  ඔහ් නේහා ජී ඔයාලාට කොහේ හරි යන්න ඕනිද
Sinhala:  [start] is only for you have a crazy do you can go[end]         
-
English:  ටොරුක් මක්ටෝ සහ පවුල අපි එක්ක ඉඳීවි
Sinhala:  [start] from [UNK] and family[end]                
-
English:  හරි
Sinhala:  [start] okay[end]                   
-
English:  මම කැමති වුනේ නෑ ඔයා අපි ගැන රේඩියෝවෙන් කියනවාට
Sinhala:  [start] i never how you would tell about us[end]            
-
English:  

In [29]:
# Define the vocabulary and index lookup for Sinhala
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

# Get English input from the user
input_sentence = input("Enter an English sentence: ")

# Translate the input sentence to Sinhala
translated_sentence = decode_sequence(input_sentence)

# Print the translated sentence
print("Sinhala translation:", translated_sentence)

Enter an English sentence: හරි එන්න අපි යමු
Sinhala translation: [start] come on lets go[end]                
